# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 14 2023 1:13PM,264754,10,9572362,Eywa Pharma Inc.,Released
1,Jun 14 2023 1:13PM,264754,10,9572363,Eywa Pharma Inc.,Released
2,Jun 14 2023 12:50PM,264752,15,30017246,"Alliance Pharma, Inc.",Released
3,Jun 14 2023 12:50PM,264752,15,WT PNC459244A(8010403) Out,"Person & Covey, Inc.",Executing
4,Jun 14 2023 12:50PM,264752,15,WT PNC459252A(8010408) Out,"Person & Covey, Inc.",Released
5,Jun 14 2023 12:50PM,264752,15,WT PNC459259A(8010412) Out,"Person & Covey, Inc.",Released
6,Jun 14 2023 12:50PM,264752,15,PNC459242A,"Person & Covey, Inc.",Released
7,Jun 14 2023 12:50PM,264752,15,PNC459243A,"Person & Covey, Inc.",Released
8,Jun 14 2023 12:50PM,264752,15,PNC459247A,"Person & Covey, Inc.",Released
9,Jun 14 2023 12:50PM,264752,15,PNC459277A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,264749,Released,1
19,264750,Released,3
20,264752,Executing,6
21,264752,Released,17
22,264754,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264748,NaN,NaN,1.0
264749,NaN,NaN,1.0
264750,NaN,NaN,3.0
264752,NaN,6.0,17.0
264754,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264703,0.0,0.0,1.0
264707,8.0,11.0,0.0
264711,0.0,0.0,1.0
264713,0.0,0.0,2.0
264721,0.0,1.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264703,0,0,1
264707,8,11,0
264711,0,0,1
264713,0,0,2
264721,0,1,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264703,0,0,1
1,264707,8,11,0
2,264711,0,0,1
3,264713,0,0,2
4,264721,0,1,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264703,,,1
1,264707,8,11,
2,264711,,,1
3,264713,,,2
4,264721,,1,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 14 2023 1:13PM,264754,10,Eywa Pharma Inc.
2,Jun 14 2023 12:50PM,264752,15,"Alliance Pharma, Inc."
3,Jun 14 2023 12:50PM,264752,15,"Person & Covey, Inc."
25,Jun 14 2023 12:14PM,264750,10,"Methapharm, Inc."
28,Jun 14 2023 12:14PM,264749,10,"Methapharm, Inc."
29,Jun 14 2023 12:13PM,264748,19,"GUSA Granules USA, Inc."
30,Jun 14 2023 12:04PM,264747,10,"Citieffe, Inc."
32,Jun 14 2023 11:18AM,264746,21,"NBTY Global, Inc."
33,Jun 14 2023 10:47AM,264744,19,"GUSA Granules USA, Inc."
34,Jun 14 2023 10:36AM,264742,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 14 2023 1:13PM,264754,10,Eywa Pharma Inc.,,,2
1,Jun 14 2023 12:50PM,264752,15,"Alliance Pharma, Inc.",,6,17
2,Jun 14 2023 12:50PM,264752,15,"Person & Covey, Inc.",,6,17
3,Jun 14 2023 12:14PM,264750,10,"Methapharm, Inc.",,,3
4,Jun 14 2023 12:14PM,264749,10,"Methapharm, Inc.",,,1
5,Jun 14 2023 12:13PM,264748,19,"GUSA Granules USA, Inc.",,,1
6,Jun 14 2023 12:04PM,264747,10,"Citieffe, Inc.",,,2
7,Jun 14 2023 11:18AM,264746,21,"NBTY Global, Inc.",,,1
8,Jun 14 2023 10:47AM,264744,19,"GUSA Granules USA, Inc.",,,1
9,Jun 14 2023 10:36AM,264742,19,"GUSA Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2023 1:13PM,264754,10,Eywa Pharma Inc.,2,,
1,Jun 14 2023 12:50PM,264752,15,"Alliance Pharma, Inc.",17,6,
2,Jun 14 2023 12:50PM,264752,15,"Person & Covey, Inc.",17,6,
3,Jun 14 2023 12:14PM,264750,10,"Methapharm, Inc.",3,,
4,Jun 14 2023 12:14PM,264749,10,"Methapharm, Inc.",1,,
5,Jun 14 2023 12:13PM,264748,19,"GUSA Granules USA, Inc.",1,,
6,Jun 14 2023 12:04PM,264747,10,"Citieffe, Inc.",2,,
7,Jun 14 2023 11:18AM,264746,21,"NBTY Global, Inc.",1,,
8,Jun 14 2023 10:47AM,264744,19,"GUSA Granules USA, Inc.",1,,
9,Jun 14 2023 10:36AM,264742,19,"GUSA Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2023 1:13PM,264754,10,Eywa Pharma Inc.,2,,
1,Jun 14 2023 12:50PM,264752,15,"Alliance Pharma, Inc.",17,6,
2,Jun 14 2023 12:50PM,264752,15,"Person & Covey, Inc.",17,6,
3,Jun 14 2023 12:14PM,264750,10,"Methapharm, Inc.",3,,
4,Jun 14 2023 12:14PM,264749,10,"Methapharm, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2023 1:13PM,264754,10,Eywa Pharma Inc.,2.0,NaN,NaN
1,Jun 14 2023 12:50PM,264752,15,"Alliance Pharma, Inc.",17.0,6.0,NaN
2,Jun 14 2023 12:50PM,264752,15,"Person & Covey, Inc.",17.0,6.0,NaN
3,Jun 14 2023 12:14PM,264750,10,"Methapharm, Inc.",3.0,NaN,NaN
4,Jun 14 2023 12:14PM,264749,10,"Methapharm, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2023 1:13PM,264754,10,Eywa Pharma Inc.,2.0,0.0,0.0
1,Jun 14 2023 12:50PM,264752,15,"Alliance Pharma, Inc.",17.0,6.0,0.0
2,Jun 14 2023 12:50PM,264752,15,"Person & Covey, Inc.",17.0,6.0,0.0
3,Jun 14 2023 12:14PM,264750,10,"Methapharm, Inc.",3.0,0.0,0.0
4,Jun 14 2023 12:14PM,264749,10,"Methapharm, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2647353,38.0,1.0,0.0
15,529504,34.0,12.0,0.0
19,1588384,5.0,11.0,8.0
20,529439,5.0,0.0,0.0
21,264746,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2647353,38.0,1.0,0.0
1,15,529504,34.0,12.0,0.0
2,19,1588384,5.0,11.0,8.0
3,20,529439,5.0,0.0,0.0
4,21,264746,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,38.0,1.0,0.0
1,15,34.0,12.0,0.0
2,19,5.0,11.0,8.0
3,20,5.0,0.0,0.0
4,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,38.0
1,15,Released,34.0
2,19,Released,5.0
3,20,Released,5.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,0.0,0.0,8.0,0.0,0.0
Executing,1.0,12.0,11.0,0.0,0.0
Released,38.0,34.0,5.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,0.0,0.0,8.0,0.0,0.0
1,Executing,1.0,12.0,11.0,0.0,0.0
2,Released,38.0,34.0,5.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,0.0,0.0,8.0,0.0,0.0
1,Executing,1.0,12.0,11.0,0.0,0.0
2,Released,38.0,34.0,5.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()